# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
# Loading CSV file
weather_data_df = pd.read_csv('Datafolder/weather_data_df.csv')
weather_data_df.head(50)
weather_data_df.dtypes

city           object
lat           float64
lon           float64
temp          float64
humidity        int64
wind          float64
cloudiness      int64
country        object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
# Store latitude and longitude in locations
locations = weather_data_df[["lat", "lon"]]

# Convert rating to float
rating = weather_data_df["humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
 
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
hotel_df = weather_data_df.loc[(weather_data_df["temp"] >= 70) &
                                                 (weather_data_df["temp"] <= 95) &
                                                 (weather_data_df["cloudiness"] < 50) &
                                                 (weather_data_df["humidity"] < 75) &
                                                 (weather_data_df["wind"] < 10)]
hotel_df.shape

(64, 8)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\jeoda\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,lat,lon,temp,humidity,wind,cloudiness,country,Hotel Name
3,Lock Haven,41.14,-77.45,87.01,43,3.00,45,US,
23,Chaparral,3.72,-75.48,78.19,55,1.79,2,CO,
33,Mbuji-Mayi,-6.15,23.60,72.07,31,1.41,0,CD,
36,Carlsbad,33.16,-117.35,78.01,64,9.17,1,US,
38,Bethel,41.37,-73.41,84.20,54,9.17,1,US,


In [56]:
# params dictionary to update each iteration
params = {
    "radius": 5000,  ## this is in meters
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


In [57]:
# Use the lat/lon
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


C:\Users\jeoda\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [58]:
# filtered_hotel_df = hotel_df.dropna(how="any")
# filtered_hotel_df

filtered_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "", :]
filtered_hotel_df = filtered_hotel_df[["city", "lat", "lon", "humidity", "country",
                            "Hotel Name"]]

filtered_hotel_df.head()

,city,lat,lon,humidity,country,Hotel Name
3,Lock Haven,41.14,-77.45,43,US,Fairfield Inn & Suites by Marriott Lock Haven
23,Chaparral,3.72,-75.48,55,CO,HOTEL LOS TORRES
33,Mbuji-Mayi,-6.15,23.60,31,CD,Hotel Equinox
36,Carlsbad,33.16,-117.35,64,US,Hilton Garden Inn Carlsbad Beach
38,Bethel,41.37,-73.41,54,US,Courtyard by Marriott Danbury


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in filtered_hotel_df.iterrows()]
locations = filtered_hotel_df[["lat", "lon"]]

In [68]:
fig = gmaps.figure()

# marker_locations = filtered_hotel_df[['lat', 'lon']]

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Convert rating to float
rating = filtered_hotel_df["humidity"]
# locations = filtered_hotel_df[["lat", "lon"]]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
 
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…